# Feature Extraction using Unsupervised Methods

This notebook augments the X_train and X_test dataframes with additional features from PCA extraction, MDS, and K-Means clustering techniques.

Additionally, it preserves all objects as python 'pickle' files so that results can be replicated in the future.

In [1]:
#123456789012345678901234567890123456789012345678901234567890123456789012345678

In [2]:
import sys
PATH_TO_MODULES = '../../..'
sys.path.insert(0, PATH_TO_MODULES)

from helpers import *
from unsupervised_helpers import *

In [3]:


def extract_features(input_df: pd.DataFrame, pickle_path='pickles/') -> pd.DataFrame:
    """Extract features from a training dataset using PCA.

    Use this function only on the TRAINING set.
    This function extracts features using PCA and assigns cluster labels
    using K-Means.  It also saves an augmented dataframe as a CSV file, and 
    preserves the objects used to transform the data as 'pickle' files.
    """
    dataset = input_df.copy()
    if 'Unnamed: 0' in dataset.columns:
        dataset.drop(columns=['Unnamed: 0'], inplace=True)
    # Process for PCA
    pca_df = process_for_PCA(dataset)
    # We found that the top 50 PCs explain 85% of the variance
    pca = PCA(50)
    X_PCA = pca.fit_transform(pca_df)
    # Pickle the pca object
    with open(pickle_path + 'pca_all.pickle', 'wb') as f:
        pickle.dump(pca, f)
    # Make a dataframe of the top 50 PCs
    PCA_cols = ['PCA_all_PC' + str(i + 1) for i in range(X_PCA.shape[1])]
    top50_PC_all_df = pd.DataFrame(X_PCA, columns=PCA_cols)
    # Perform PCA using only the KPI subset
    subset_df = get_KPI(dataset)
    subset_pca_df = process_for_PCA(subset_df)
    pca_KPI = PCA(10)
    X_PCA_KPI = pca_KPI.fit_transform(pca_df)
    # pickle the pca_KPI object
    with open(pickle_path + 'pca_KPI.pickle', 'wb') as f:
        pickle.dump(pca_KPI, f)
    # Make a dataframe of the top 10 PCs from the KPI subset
    PCA_KPI_cols = ['PCA_KPI_PC' + str(i + 1) for i in range(X_PCA_KPI.shape[1])]
    top10_PC_KPI_df = pd.DataFrame(X_PCA_KPI, columns=PCA_KPI_cols)
    # Use K-Means to assign cluster labels
    kmeans = KMeans(n_clusters=7, init='random', n_init=100, copy_x=False)
    cluster = kmeans.fit_predict(X_PCA_KPI)
    # Pickle the K-Means object
    with open(pickle_path + 'kmeans.pickle', 'wb') as f:
        pickle.dump(kmeans, f)
    # Create augmented DataFrame
    augmented_df = pd.concat(
        (
            dataset,
            top50_PC_all_df,
            top10_PC_KPI_df,
            pd.Series(cluster, name='Cluster')
        ), axis = 1
    )
    return augmented_df

    
def transform_features(input_df: pd.DataFrame) -> pd.DataFrame:
    """Augment a dataframe with extracted features.

    Use this function on the test set.
    This function uses pickled objects that were used during feature extraction
    on the test set to augment the test set with additional features.
    """
    dataset = input_df.copy()
    if 'Unnamed: 0' in dataset.columns:
        dataset.drop(columns=['Unnamed: 0'], inplace=True)
    # Need to finish this....
    return dataset
    
    


## Load data


In [4]:
# Import merged dataset
path = PATH_TO_MODULES + '/datasets/'
input_filename = 'X_train_filled_KPIs_QoQ.csv'
dataset = pd.read_csv(path + input_filename)
print(f'There are {dataset.shape[0]} rows and {dataset.shape[1]} columns in the dataset.')

There are 1910 rows and 265 columns in the dataset.


In [6]:
pickle_path = PATH_TO_MODULES + '/pickles/'
augmented_df = extract_features(dataset, pickle_path=pickle_path)
augmented_df

Ticker                               Name                  Sector  \
0      ACIW                  ACI WORLDWIDE INC  Information Technology   
1      HONE              HARBORONE BANCORP INC              Financials   
2      REPL                REPLIMUNE GROUP INC             Health Care   
3      RBRK                 RUBRIK INC CLASS A  Information Technology   
4       CSL             CARLISLE COMPANIES INC             Industrials   
...     ...                                ...                     ...   
1905    CNS                 COHEN & STEERS INC              Financials   
1906    FBP                      FIRST BANCORP              Financials   
1907   RDDT                 REDDIT INC CLASS A           Communication   
1908    AGM  FEDERAL AGRICULTURAL MORTGAGE NON              Financials   
1909    EAT          BRINKER INTERNATIONAL INC  Consumer Discretionary   

      CapitalExpenditure_2024Q2  CapitalExpenditure_2024Q3  \
0                    -3777000.0                 -4045500.0   
1                     -220000.0                  -569000.0   
2                    -1618000.0                 -1114000.0   
3                   -15766000.0                 -4929000.0   
4                   -24900000.0                -19300000.0   
...                         ...                        ...   
1905                 -4239000.0                 -1408000.0   
1906                 -3264000.0                 -2547000.0   
1907                 -1202000.0                 -1353000.0   
1908                 -3568000.0                   -66000.0   
1909                -58000000.0                -56500000.0   

      CapitalExpenditure_2024Q4  CapitalExpenditure_2025Q1  \
0                    -4663500.0                 -4112000.0   
1                     -339000.0                  -208000.0   
2                    -2266000.0                 -1503000.0   
3                    -7527000.0                 -8401000.0   
4                   -36600000.0                -29000000.0   
...                         ...                        ...   
1905                 -1678000.0                 -1075000.0   
1906                 -3407000.0                 -3396000.0   
1907                  -842000.0                  -979000.0   
1908                 -3407000.0                 -3396000.0   
1909                -49300000.0                -79600000.0   

      CashAndSTInvestments_2024Q2  CashAndSTInvestments_2024Q3  \
0                    1.851080e+08                 1.970750e+08   
1                    2.350620e+08                 2.242790e+08   
2                    8.278500e+07                 7.524700e+07   
3                    4.293020e+08                 1.423490e+08   
4                    1.736300e+09                 1.530600e+09   
...                           ...                          ...   
1905                 1.220130e+08                 1.064740e+08   
1906                 4.240230e+08                 5.029450e+08   
1907                 4.679520e+08                 5.158950e+08   
1908                 9.229610e+08                 8.420560e+08   
1909                 6.460000e+07                 1.620000e+07   

      CashAndSTInvestments_2024Q4  ...  PCA_KPI_PC2  PCA_KPI_PC3  PCA_KPI_PC4  \
0                    1.883640e+08  ...    -0.123216     0.554203     0.635451   
1                    2.310710e+08  ...     1.653149    -1.471926    -0.344523   
2                    7.830350e+07  ...    -0.800500     0.397646    -0.933726   
3                    1.038960e+08  ...    -1.510564    -0.243129    -2.950378   
4                    7.535000e+08  ...    -0.909721     0.196802    -0.543445   
...                           ...  ...          ...          ...          ...   
1905                 1.829740e+08  ...     0.866749     2.232555     0.594781   
1906                 5.027000e+08  ...     1.570123    -0.617818    -0.528226   
1907                 5.620920e+08  ...     0.095268     2.070966    -0.487123   
1908                 1.